# SP500 Stock Demo — Notebook 02: Feature Store

- Compute core technical features on simulated hourly data
- Register Feature Store `Entity(TICKER)` and `FeatureView(price_features)`
- Save to `SP500_STOCK_DEMO.DATA`


In [ ]:
# 0) Imports and session
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, avg, stddev, sqrt, lag, when
from snowflake.snowpark import Window
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode

session = get_active_session()
session.sql("USE DATABASE SP500_STOCK_DEMO").collect()
session.sql("USE SCHEMA DATA").collect()
session.sql("USE WAREHOUSE DEMO_WH_M").collect()

session.table('HOURLY_SP500_SIM').limit(5).show()


In [ ]:
# 1) Feature engineering with Snowpark (join hourly + static mapping)
win_5 = Window.partition_by('SYMBOL').order_by(col('TS')).rows_between(-4, 0)
win_20 = Window.partition_by('SYMBOL').order_by(col('TS')).rows_between(-19, 0)

hourly = session.table('HOURLY_SP500_SIM')
spmap = session.table('SP_500_LIST').select(col('SYMBOL').alias('MAP_SYMBOL'), col('SECTOR'))

# Join to enrich features with a static attribute (e.g., sector)
enriched = (
    hourly.join(spmap, hourly['SYMBOL'] == spmap['MAP_SYMBOL'], how='left')
          .drop('MAP_SYMBOL')
)

features = (
    enriched
    .with_column('RET_1', (col('CLOSE')/lag(col('CLOSE'), 1).over(win_5) - 1))
    .with_column('SMA_5', avg(col('CLOSE')).over(win_5))
    .with_column('SMA_20', avg(col('CLOSE')).over(win_20))
    .with_column('VOL_20', stddev(col('CLOSE')).over(win_20))
    .with_column('RSI_PROXY', when(col('RET_1')>0, col('RET_1')).otherwise(0))
    .select('SYMBOL','SECTOR','TS','CLOSE','VOLUME','RET_1','SMA_5','SMA_20','VOL_20','RSI_PROXY')
)

features.write.save_as_table('PRICE_FEATURES', mode='overwrite')
features.limit(5).show()


In [ ]:
# 2) Demonstrate Feature Store joining: spine_df + FeatureView
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode

fs = FeatureStore(
    session=session,
    database='SP500_STOCK_DEMO',
    name='DATA',
    default_warehouse='DEMO_WH_M',
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

# Update entity to include SYMBOL join key and demonstrate a spine
TICKER = Entity(name='TICKER', join_keys=['SYMBOL'])
fs.register_entity(TICKER)

fv = FeatureView(
    name='price_features',
    entities=[TICKER],
    feature_df=session.table('PRICE_FEATURES'),
    desc='Hourly price features with sector enrichment'
)
fs.register_feature_view(feature_view=fv, version='V2', overwrite=True)

# Spine dataframe (e.g., a list of S&P500 symbols and a time boundary)
spine_df = session.table('SP_500_LIST').select(col('SYMBOL')).limit(50)
# Retrieve features onto the spine (point-in-time join not demonstrated here)
joined = fs.retrieve_feature_values(spine_df=spine_df, features=[fv])
joined.limit(5).show()


In [ ]:
# 3) Register/verify V1 (legacy) — optional
fs = FeatureStore(
    session=session,
    database='SP500_STOCK_DEMO',
    name='DATA',
    default_warehouse='DEMO_WH_M',
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

TICKER = Entity(name='TICKER', join_keys=['TICKER'])
fs.register_entity(TICKER)

fv_v1 = FeatureView(
    name='price_features',
    entities=[TICKER],
    feature_df=session.table('PRICE_FEATURES'),
    desc='Hourly price features with sector enrichment'
)
fs.register_feature_view(
    feature_view=fv_v1,
    version='V2',
    overwrite=True
)

fs.list_feature_views().show()
